# Bayesian model

So that's cool, we can run the SIR model forward given parameters and get predictions depending on our scenario.

Now we want to use publically available data (deaths per day) to estimate those parameters from data.

We can create a bayesian hierarchical model for this:

* $deaths[i]$: observed variable, daily deaths, distributed depending on infection fatality rate and daily recoveries: $P(deaths[i] \mid \alpha, recovered[i])$
* $recovered(i, s_0, \beta_0, \beta_1, t\_lock, \gamma)$: recovered at day $i$, depends deterministically on our model
* $\alpha$ infection fatality rate
* $s_0$ SIR state for day 0
* $\beta_0$ SIR model beta for no-lockdown days
* $\beta_1$ SIR model beta for lockdown days
* $t\_lock$: point in time where lockdown measures are implemented
* $\gamma$ SIR model gamma, independent of lockdown


The distribution over the parameters is proportional to (assuming uniform priors over parameters to avoid writing their priors):

$$
P(\alpha, s_0, \beta_0, \beta_1, \gamma) \propto \prod_i{P(deaths[i] \mid \alpha, recovered(i, s_0, \beta_0, \beta_1, t\_lock, \gamma))}
$$

## $P(deaths[i] \mid \alpha, recovered[i])$

We're going to use the [gaussian approximation to the binomial](https://en.wikipedia.org/wiki/Binomial_distribution#Normal_approximation) and say the number of deaths per day follows this distribution, conditional on the number of recoveries per day and the infection fatality rate:

$$
deaths[i] \sim \mathcal{N}(recovered[i] \cdot \alpha, recovered[i]\cdot \alpha \cdot (1-\alpha)))
$$

## Inference problem specification

To fully characterize an inference problem that will use this model, we need the following parameters (copying the docs from the `Spec` class that represents a problem specification):

```
population : float
    Total population for the region being studied
deaths : ndarray
    daily death counts due to disease
b_infected : Tuple[float]
    lower and upper bounds on the possible infected population on day 0
b_alpha : Tuple[float]
    lu bounds on infection fatality rate
b_beta_0 : Tuple[float]
    lu bounds on SIR beta prior to lockdown
b_beta_1 : Tuple[float]
    lu bounds on SIR beta after lockdown
t_lock : float
    time lockdown went into effect
b_gamma : Tuple[float]
    lu bounds on SIR gamma
```